In [ ]:
from psychopy import visual, core, logging, event as pevent
from psychopy.hardware.emulator import launchScan
import time
import serial

In [ ]:
event = pevent

class SerialEvent(object):
    """Extends psychopy.event to get input for serial port"""
    def __init__(self):
        """Open serial port"""
        # set up an internal clock
        # self.clock = core.Clock()
        self.ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=.0001)
        self.ser.flushInput()

    def getKeys(self, keyList=None, timeStamped=False):
        """Use same arguments as event.getKeys"""
        keys = []
        serkey = None
        if SYNCKEY in keyList:
            # XXX: it can only send one key at a time
            serkey = self.ser.read()
            if serkey == SYNCKEY:
                keys.append((serkey, core.getTime()))
                keyList.remove(SYNCKEY)
        if RESPONSEKEY in keyList:
            serkey = self.ser.read() if not serkey else serkey
            if serkey == RESPONSEKEY:
                keys.append((serkey, core.getTime()))
                keyList.remove(RESPONSEKEY)
        if keyList is None or keyList:
            keys.extend(pevent.getKeys(keyList=keyList,
                                       timeStamped=True))
        # code borrowed from psychopy.event
        if not timeStamped:
            keyNames = [k[0] for k in keys]
            return keyNames
        elif hasattr(timeStamped, 'getLastResetTime'):
            #keys were originally time-stamped with core.monotonicClock._lastResetTime
            #we need to shift that by the difference between it and our custom clock
            timeBaseDiff = timeStamped.getLastResetTime() - core.monotonicClock.getLastResetTime()
            relTuple = [(k[0], k[1]-timeBaseDiff) for k in keys]
            return relTuple
        elif timeStamped:
            return keys
        elif isinstance(timeStamped, (float, int, long)):
            relTuple = [(k[0], k[1] - timeStamped) for k in keys]
            return relTuple

    @staticmethod
    def waitKeys(*args, **kwargs):
        return pevent.waitKeys(*args, **kwargs)

    def close(self):
        self.ser.close()
